<a href="https://colab.research.google.com/github/iam-fern/Generate_data/blob/main/ex_sampling_c_100rep_n50_x6_sas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

import statistics as stats
import scipy.stats as ss
import scipy.stats as st
from scipy.stats import f_oneway
from scipy import stats
import scipy.stats as stats
import time
import statsmodels.api as sm

from statsmodels.formula.api import ols

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df = pd.read_csv('https://github.com/iam-fern/Research/raw/main/sampling_100rep_n50_x6_sas.csv', index_col=['rep', 'index'])
df

x1         x2  ...         y_hat         e_hat
rep  index                        ...                            
0.0  0.0    38.074881  47.501957  ... -1.415257e+08  2.305435e+09
     1.0    47.722055  25.548748  ... -9.591188e+06  8.956481e+09
     2.0    32.936625  33.592004  ... -1.340543e+08 -3.449127e+07
     3.0    28.597172  34.693307  ...  2.007092e+07  4.484362e+08
     4.0    32.452230  49.545322  ... -2.861253e+08 -1.107651e+09
...               ...        ...  ...           ...           ...
99.0 45.0   40.855659  34.206762  ...  5.735301e+06  4.943821e+09
     46.0   49.488871  11.873135  ... -1.285755e+08  2.993224e+09
     47.0   47.625297  10.268319  ...  1.669600e+08  1.308306e+10
     48.0   11.232647  24.371463  ...  5.220381e+07 -5.091323e+07
     49.0   40.629392  44.215701  ... -2.237210e+08  6.557172e+08

[5000 rows x 7 columns]

In [3]:
# ลบข้อมูลที่มี missing values
table = df.dropna()
table

x1         x2  ...         y_hat         e_hat
rep  index                        ...                            
0.0  0.0    38.074881  47.501957  ... -1.415257e+08  2.305435e+09
     1.0    47.722055  25.548748  ... -9.591188e+06  8.956481e+09
     2.0    32.936625  33.592004  ... -1.340543e+08 -3.449127e+07
     3.0    28.597172  34.693307  ...  2.007092e+07  4.484362e+08
     4.0    32.452230  49.545322  ... -2.861253e+08 -1.107651e+09
...               ...        ...  ...           ...           ...
99.0 45.0   40.855659  34.206762  ...  5.735301e+06  4.943821e+09
     46.0   49.488871  11.873135  ... -1.285755e+08  2.993224e+09
     47.0   47.625297  10.268319  ...  1.669600e+08  1.308306e+10
     48.0   11.232647  24.371463  ...  5.220381e+07 -5.091323e+07
     49.0   40.629392  44.215701  ... -2.237210e+08  6.557172e+08

[5000 rows x 7 columns]

In [4]:
table.reset_index(inplace=True)
table

,rep,index,x1,x2,x3,e,Y,y_hat,e_hat
0,0.0,0.0,38.074881,47.501957,29.305364,2.163909e+09,2.163909e+09,-1.415257e+08,2.305435e+09
1,0.0,1.0,47.722055,25.548748,34.240754,8.946889e+09,8.946890e+09,-9.591188e+06,8.956481e+09
2,0.0,2.0,32.936625,33.592004,19.476092,-1.685459e+08,-1.685455e+08,-1.340543e+08,-3.449127e+07
3,0.0,3.0,28.597172,34.693307,38.034853,4.685067e+08,4.685071e+08,2.007092e+07,4.484362e+08
4,0.0,4.0,32.452230,49.545322,10.918857,-1.393777e+09,-1.393776e+09,-2.861253e+08,-1.107651e+09
...,...,...,...,...,...,...,...,...,...
4995,99.0,45.0,40.855659,34.206762,39.727211,4.949556e+09,4.949557e+09,5.735301e+06,4.943821e+09
4996,99.0,46.0,49.488871,11.873135,10.970369,2.864648e+09,2.864649e+09,-1.285755e+08,2.993224e+09
4997,99.0,47.0,47.625297,10.268319,46.107398,1.325001e+10,1.325002e+10,1.669600e+08,1.308306e+10
4998,99.0,48.0,11.232647,24.371463,29.852099,1.290407e+06,1.290578e+06,5.220381e+07,-5.091323e+07


In [5]:
table.set_index(['rep', 'index'], inplace=True)

In [6]:
table

x1         x2  ...         y_hat         e_hat
rep  index                        ...                            
0.0  0.0    38.074881  47.501957  ... -1.415257e+08  2.305435e+09
     1.0    47.722055  25.548748  ... -9.591188e+06  8.956481e+09
     2.0    32.936625  33.592004  ... -1.340543e+08 -3.449127e+07
     3.0    28.597172  34.693307  ...  2.007092e+07  4.484362e+08
     4.0    32.452230  49.545322  ... -2.861253e+08 -1.107651e+09
...               ...        ...  ...           ...           ...
99.0 45.0   40.855659  34.206762  ...  5.735301e+06  4.943821e+09
     46.0   49.488871  11.873135  ... -1.285755e+08  2.993224e+09
     47.0   47.625297  10.268319  ...  1.669600e+08  1.308306e+10
     48.0   11.232647  24.371463  ...  5.220381e+07 -5.091323e+07
     49.0   40.629392  44.215701  ... -2.237210e+08  6.557172e+08

[5000 rows x 7 columns]

In [7]:
def get_result(run_replicate):
    new_table = table.loc[run_replicate]

    # show index
    new_table.reset_index(inplace=True)

    new_table.drop(['index'], axis=1, inplace=True)


    # แบ่งข้อมูลออกเป็น 2 กลุ่มด้วย k-mean

    #Set number of clusters
    x = new_table[["x1", "x2", "x3"]]
    k_means = KMeans(n_clusters=2)

    #Run the clustering algorithm
    model = k_means.fit(x)
    model

    #Generate cluster predictions and store in y_kmeans
    y_kmeans = k_means.predict(x)
    y_kmeans

    # kmean_clusters.astype(float)
    kmean_clusters = pd.DataFrame({"clusters" : y_kmeans})

    # เชื่อมข้อมูลทั้ง 2 DataFrame เข้าด้วยกันด้วยคำสั่ง pd.concat
    data_clusters = pd.concat([new_table, kmean_clusters], axis=1)

    data_sort_clusters = data_clusters.sort_values('clusters')

    # print(data_clusters.groupby('clusters').groups)

    data_sort_clusters.reset_index(inplace=True)

    # กลุ่มที่ 1
    data_cluster1 = data_sort_clusters[data_sort_clusters.clusters == 1]

    y = data_cluster1[['Y']]
    x = data_cluster1[['x1', 'x2', 'x3']]
    x = sm.add_constant(x)
    model1 = sm.OLS(y, x).fit()

    DF1 = model1.df_resid

    SSR1 = model1.ssr

    MSE1 = model1.mse_resid

    # กลุ่มที่ 2
    data_cluster0 = data_sort_clusters[data_sort_clusters.clusters == 0]

    y = data_cluster0[['Y']]
    x = data_cluster0[['x1', 'x2', 'x3']]
    x = sm.add_constant(x)
    model0 = sm.OLS(y, x).fit()

    DF0 = model0.df_resid

    SSR0 = model0.ssr

    MSE0 = model0.mse_resid

    df1 = DF1+1
    df0 = DF0+1

    # คำนวณค่าสถิติ
    GQ_test_clusters = (SSR1/df1) / (SSR0/df0)


    # คำนวณค่า p-value
    # sf(x, dfn, dfd, loc=0, scale=1) : Survival function (also defined as 1 - cdf, but sf is sometimes more accurate)
    two_tailed = 2.0*(1.0-(ss.f.cdf(GQ_test_clusters, df1, df0)))

    # pdf(x, dfn, dfd, loc=0, scale=1) : Probability density function
    p_greater = ss.f.cdf(GQ_test_clusters, df1, df0)
    
    return df0, df1, SSR0, SSR1, MSE0, MSE1, GQ_test_clusters, two_tailed, p_greater

# สร้าง list ของ replicates
list_replicate=list({i[0] for i in table.index})
list_replicate.sort()

# สร้าง list เพื่อเก็บค่า ทุกรอบที่ run
list_df0=[]
list_df1=[]
list_SSR0=[]
list_SSR1=[]
list_MSE0=[]
list_MSE1=[]
list_GQ_test_clusters=[]
list_result_test=[]
list_two_tailed=[]
list_p_greater=[]
list_result_test01=[]

# ใช้ for loop สำหรับการ run ทุก replicates
for run_replicate in list_replicate:
    df0, df1, SSR0, SSR1, MSE0, MSE1, GQ_test_clusters, two_tailed, p_greater=get_result(run_replicate)

    list_df0.append(df0)
    list_df1.append(df1)
    list_SSR0.append(SSR0)
    list_SSR1.append(SSR1)
    list_MSE0.append(MSE0)
    list_MSE1.append(MSE1)
    list_GQ_test_clusters.append(GQ_test_clusters)
    list_two_tailed.append(two_tailed)
    list_p_greater.append(p_greater)

In [8]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
two_tailed_new=pd.DataFrame({"Replicate":list_replicate, "df0":list_df0, "df1":list_df1, "SSR0":list_SSR0,
                     "SSR1":list_SSR1, "MSE0":list_MSE0, "MSE1":list_MSE1, 
                     "GQ_test_clusters":list_GQ_test_clusters, "two_tailed":list_two_tailed, "greater":list_p_greater})
two_tailed_new.set_index(['Replicate'], inplace=True)
two_tailed_new=two_tailed_new[(two_tailed_new["two_tailed"]<=1) & (two_tailed_new["greater"]<=1)]
two_tailed_new["Result_test"]=two_tailed_new["two_tailed"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                  if p_value < alpha else 'Reject005=1 : Homoscedasticity')
#two_tailed_new_show=two_tailed_new.drop(columns="greater")
two_tailed_new.astype(str)

,df0,df1,SSR0,SSR1,MSE0,MSE1,GQ_test_clusters,two_tailed,greater,Result_test
Replicate,,,,,,,,,,
1.0,15.0,29.0,8.0728940325816e+19,4.0162282808896225e+20,5.766352880415428e+18,1.434367243174865e+19,2.5732524758141198,0.05708763923866633,0.9714561803806668,Reject005=1 : Homoscedasticity
2.0,31.0,13.0,2.345756566781653e+20,2.8886488387645066e+20,7.819188555938844e+18,2.407207365637089e+19,2.9365009819070385,0.013742416307551908,0.993128791846224,Reject005=0 : Heteroscedasticity
4.0,19.0,25.0,2.1111925797278007e+20,4.2946431154464614e+20,1.172884766515445e+19,1.7894346314360257e+19,1.5460118603486823,0.33325737532162214,0.8333713123391889,Reject005=1 : Homoscedasticity
5.0,17.0,27.0,4.468100886250444e+19,6.963920592565684e+20,2.7925630539065277e+18,2.678430997140648e+19,9.813320815920164,1.0764069812596588e-05,0.9999946179650937,Reject005=0 : Heteroscedasticity
6.0,22.0,22.0,5.73807418260743e+20,1.3633590449212157e+21,2.7324162774321095e+19,6.492185928196265e+19,2.3759871370322605,0.04804835767819848,0.9759758211609008,Reject005=0 : Heteroscedasticity
12.0,23.0,21.0,6.747253124927888e+19,1.5088521422277532e+20,3.0669332386035855e+18,7.544260711138766e+18,2.449222395620597,0.039217474005557085,0.9803912629972215,Reject005=0 : Heteroscedasticity
13.0,20.0,24.0,3.740751584095143e+16,3.763205576443995e+20,1968816623207970.0,1.6361763375843457e+19,8383.354458420248,2.220446049250313e-16,0.9999999999999999,Reject005=0 : Heteroscedasticity
15.0,22.0,22.0,2.270229974444852e+20,4.923968396774109e+20,1.0810618925927866e+19,2.3447468556067185e+19,2.1689293385257966,0.07609842101819808,0.961950789490901,Reject005=1 : Homoscedasticity
16.0,24.0,20.0,3.434420516554921e+20,3.938167051984239e+20,1.4932263115456178e+19,2.0727195010443362e+19,1.37601101542497,0.4520007497071188,0.7739996251464406,Reject005=1 : Homoscedasticity


In [9]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
two_tailed_new=pd.DataFrame({"Replicate":list_replicate, "df0":list_df0, "df1":list_df1, "SSR0":list_SSR0,
                     "SSR1":list_SSR1, "MSE0":list_MSE0, "MSE1":list_MSE1,
                     "GQ_test_clusters":list_GQ_test_clusters, "two_tailed":list_two_tailed, "greater":list_p_greater})
two_tailed_new.set_index(['Replicate'], inplace=True)

two_tailed_new["two_tailed_new"]=two_tailed_new[["two_tailed","greater"]].apply(lambda value: value[0] if value[0]<0.5 else 2*value[1], axis=1)
two_tailed_new["Result_test"]=two_tailed_new["two_tailed_new"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                      if p_value < alpha else 'Reject005=1 : Homoscedasticity')


two_tailed_new_show=two_tailed_new.drop(columns=["two_tailed", "greater"])
two_tailed_new_show.astype(str)

,df0,df1,SSR0,SSR1,MSE0,MSE1,GQ_test_clusters,two_tailed_new,Result_test
Replicate,,,,,,,,,
0.0,17.0,27.0,6.907286346887247e+20,5.186655302200845e+20,4.317053966804529e+19,1.994867423923402e+19,0.4727865174451528,0.07950315140204632,Reject005=1 : Homoscedasticity
1.0,15.0,29.0,8.0728940325816e+19,4.0162282808896225e+20,5.766352880415428e+18,1.434367243174865e+19,2.5732524758141198,0.05708763923866633,Reject005=1 : Homoscedasticity
2.0,31.0,13.0,2.345756566781653e+20,2.8886488387645066e+20,7.819188555938844e+18,2.407207365637089e+19,2.9365009819070385,0.013742416307551908,Reject005=0 : Heteroscedasticity
3.0,21.0,23.0,8.223570347056773e+20,4.934260758885351e+20,4.111785173528387e+19,2.242845799493341e+19,0.5478392493536842,0.16259278907802677,Reject005=1 : Homoscedasticity
4.0,19.0,25.0,2.1111925797278007e+20,4.2946431154464614e+20,1.172884766515445e+19,1.7894346314360257e+19,1.5460118603486823,0.33325737532162214,Reject005=1 : Homoscedasticity
5.0,17.0,27.0,4.468100886250444e+19,6.963920592565684e+20,2.7925630539065277e+18,2.678430997140648e+19,9.813320815920164,1.0764069812596588e-05,Reject005=0 : Heteroscedasticity
6.0,22.0,22.0,5.73807418260743e+20,1.3633590449212157e+21,2.7324162774321095e+19,6.492185928196265e+19,2.3759871370322605,0.04804835767819848,Reject005=0 : Heteroscedasticity
7.0,18.0,26.0,6.302045862766895e+20,2.5094811890306125e+20,3.707085801627586e+19,1.003792475612245e+19,0.2756776400393532,0.0028848026525394475,Reject005=0 : Heteroscedasticity
8.0,25.0,19.0,1.383573168812972e+21,1.925011132732348e+17,5.764888203387384e+19,1.069450629295749e+16,0.00018307014347114265,1.5142810522220218e-31,Reject005=0 : Heteroscedasticity


In [10]:
two_tailed_new_show['Result_test'].value_counts()

Reject005=0 : Heteroscedasticity    71
Reject005=1 : Homoscedasticity      29
Name: Result_test, dtype: int64